# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.27.0


## Initialize Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AFL3UNPJV to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-144414
aml-quickstarts-144414
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


## Create an Azure ML experiment

In [3]:
# choose a name for experiment
experiment_name = 'loan-prediction'
project_folder = './loan-prediction-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
loan-prediction,quick-starts-ws-144414,Link to Azure Machine Learning studio,Link to Documentation


## Create a compute cluster


In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "raw-loan-prediction-dataset"
description_text = "Loan prediction dataset before cleaning"

if key in ws.datasets.keys(): 
        found = True
        raw_dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/fnakashima/nd00333-capstone/master/starter_file/dataset/train_u6lujuX_CVtuZ9i.csv'
        raw_dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        raw_dataset = raw_dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = raw_dataset.to_pandas_dataframe()
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,False,0,Graduate,False,5849,0.0,NaN,360.0,1.0,Urban,True
1,LP001003,Male,True,1,Graduate,False,4583,1508.0,128.0,360.0,1.0,Rural,False
2,LP001005,Male,True,0,Graduate,True,3000,0.0,66.0,360.0,1.0,Urban,True
3,LP001006,Male,True,0,Not Graduate,False,2583,2358.0,120.0,360.0,1.0,Urban,True
4,LP001008,Male,False,0,Graduate,False,6000,0.0,141.0,360.0,1.0,Urban,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,False,0,Graduate,False,2900,0.0,71.0,360.0,1.0,Rural,True
610,LP002979,Male,True,3+,Graduate,False,4106,0.0,40.0,180.0,1.0,Rural,True
611,LP002983,Male,True,1,Graduate,False,8072,240.0,253.0,360.0,1.0,Urban,True
612,LP002984,Male,True,2,Graduate,False,7583,0.0,187.0,360.0,1.0,Urban,True


## Clean dataset

In [7]:
def clean_data(data):
    # Dict for cleaning data
    dependents = {"0":0, "1":1, "2":2, "3+":3}
    property_areas = {"Urban":1, "Semiurban":2, "Rural":3}

    # Clean and one hot encode data
    x_df = data.dropna()
    x_df.drop("Loan_ID", axis=1, inplace=True)

    # Filtering "True", "Yes", "Y" won't work as it will be recoginised as a boolean value automatically by dataset framework
    x_df.loc[:,('Gender')] = x_df.Gender.apply(lambda s: 1 if s == "Male" else 2)
    x_df.loc[:,('Married')] = x_df.Married.apply(lambda s: 1 if s else 0)
    x_df.loc[:,('Dependents')] = x_df.Dependents.map(dependents)
    x_df.loc[:,('Education')] = x_df.Education.apply(lambda s: 1 if s == "Graduate" else 0)
    x_df.loc[:,('Self_Employed')] = x_df.Self_Employed.apply(lambda s: 1 if s else 0)
    x_df.loc[:,('Property_Area')] = x_df.Property_Area.map(property_areas)

    y_df = x_df.pop("Loan_Status").apply(lambda s: 1 if s else 0)
    return x_df, y_df

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd

df
x, y = clean_data(df)
x

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
1,1,1,1,1,0,4583,1508.0,128.0,360.0,1.0,3
2,1,1,0,1,1,3000,0.0,66.0,360.0,1.0,1
3,1,1,0,0,0,2583,2358.0,120.0,360.0,1.0,1
4,1,0,0,1,0,6000,0.0,141.0,360.0,1.0,1
5,1,1,2,1,1,5417,4196.0,267.0,360.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
609,2,0,0,1,0,2900,0.0,71.0,360.0,1.0,3
610,1,1,3,1,0,4106,0.0,40.0,180.0,1.0,3
611,1,1,1,1,0,8072,240.0,253.0,360.0,1.0,1
612,1,1,2,1,0,7583,0.0,187.0,360.0,1.0,1


In [9]:
y

1      0
2      1
3      1
4      1
5      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 478, dtype: int64

## Split train and test data

In [10]:
from sklearn.model_selection import train_test_split
import pandas as pd


# Split data into train and test sets.
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# Default test_size: 0.25
x_train, x_test, y_train, y_test = train_test_split(x, y)

# concatenate train data x and y and test
# https://pandas.pydata.org/docs/user_guide/merging.html
train_data = pd.concat([x_train, y_train], axis=1)
pd.DataFrame(train_data)

test_data = pd.concat([x_test, y_test], axis=1)
pd.DataFrame(test_data)

train_data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
372,1,1,2,1,1,9323,7873.0,380.0,300.0,1.0,3,1
555,1,1,1,1,0,5468,1032.0,26.0,360.0,1.0,2,1
64,2,0,0,1,0,4166,0.0,116.0,360.0,0.0,2,0
452,1,1,0,1,0,3948,1733.0,149.0,360.0,0.0,3,0
248,1,1,1,1,0,2882,1843.0,123.0,480.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
39,1,0,0,0,0,3748,1668.0,110.0,360.0,1.0,2,1
245,1,0,0,1,1,6050,4333.0,120.0,180.0,1.0,1,0
173,1,1,0,1,0,5708,5625.0,187.0,360.0,1.0,2,1
606,1,1,1,1,0,3400,2500.0,173.0,360.0,1.0,2,1


In [11]:
ds_name = "loan-prediction-dataset"
ds_description = "Loan prediction dataset after cleaning"

if key in ws.datasets.keys(): 
    found = True
    train_ds = ws.datasets[key] 
    print('Found existing train dataset, use it.')

if not found:
    datastore = ws.get_default_datastore()
    train_ds = TabularDatasetFactory.register_pandas_dataframe(train_data, datastore, ds_name, description=ds_description)


train_df = train_ds.to_pandas_dataframe()
train_df

Found existing train dataset, use it.


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,False,0,Graduate,False,5849,0.0,NaN,360.0,1.0,Urban,True
1,LP001003,Male,True,1,Graduate,False,4583,1508.0,128.0,360.0,1.0,Rural,False
2,LP001005,Male,True,0,Graduate,True,3000,0.0,66.0,360.0,1.0,Urban,True
3,LP001006,Male,True,0,Not Graduate,False,2583,2358.0,120.0,360.0,1.0,Urban,True
4,LP001008,Male,False,0,Graduate,False,6000,0.0,141.0,360.0,1.0,Urban,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,False,0,Graduate,False,2900,0.0,71.0,360.0,1.0,Rural,True
610,LP002979,Male,True,3+,Graduate,False,4106,0.0,40.0,180.0,1.0,Rural,True
611,LP002983,Male,True,1,Graduate,False,8072,240.0,253.0,360.0,1.0,Urban,True
612,LP002984,Male,True,2,Graduate,False,7583,0.0,187.0,360.0,1.0,Urban,True


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [37]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=train_ds,
                             label_column_name="Loan_Status",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [38]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
loan-prediction,AutoML_223a219a-581f-455e-9ca1-8045c9a3322b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [39]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [40]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
loan-prediction,AutoML_223a219a-581f-455e-9ca1-8045c9a3322b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*************************************************

{'runId': 'AutoML_223a219a-581f-455e-9ca1-8045c9a3322b',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-10T18:10:46.778107Z',
 'endTimeUtc': '2021-05-10T18:41:14.416644Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"loan-prediction","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-144414","workspace_name":"quick-starts-ws-144414","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classific

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [42]:
# Retrieve best model from Pipeline Run
best_automl_run, fitted_automl_model = remote_run.get_output()
print(best_automl_run)
best_automl_run_metrics = best_automl_run.get_metrics()
print(best_automl_run_metrics['AUC_weighted'])

Run(Experiment: loan-prediction,
Id: AutoML_223a219a-581f-455e-9ca1-8045c9a3322b_75,
Type: azureml.scriptrun,
Status: Completed)
0.8179503984829056


In [43]:
fitted_automl_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('61',
                                             Pipeline(memory=None,
                                                      steps=[('sparsenormalizer',
                                                              <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7ff6fd1f8080>),
                                                             ('xgboostclassifier',
                                                              XGBoostClassifier(base_score=0

In [44]:
# Retrieve and save your best automl model.
best_automl_run, fitted_automl_model = remote_run.get_output()
best_automl_run_metrics = best_automl_run.get_metrics()

automl_model = remote_run.register_model(model_name = 'loan-prediction-automl-model',
                                  tags={'Method':'AutoML'},
                                  description='AutoML Model trained on loan prediction data to predict a loan status of customers')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service